In [ ]:
import os
import re

import pandas as pd
import networkx as nx
from pyvis.network import Network

In [ ]:
FILE_PATH = os.path.join(os.path.abspath('.'))
DATA_DIR = os.path.join(FILE_PATH, 'data')

In [ ]:
characters_dataset = pd.read_csv(os.path.join(DATA_DIR, 'characters_dataset.csv'))
characters_unique = pd.read_csv(os.path.join(DATA_DIR, 'characters_unique_dataset.csv'))

In [ ]:
#Limpeza do characters_dataset de valores terminando com []
relative = []
relationship = []

#Limpeza para a columa relative
for i in characters_dataset['relative'].tolist():
    
    if i.find('[') != -1:
        relative.append(i[:i.find('[')])
    else: 
        relative.append(i)

#Limpeza para a columa relationship
for y in characters_dataset['relationship'].tolist():
    
    if y.find('[') != -1:
        relationship.append(y[:y.find('[')])
    else: 
        relationship.append(y)
        
characters_dataset['relative'] = relative
characters_dataset['relationship'] = relationship

In [ ]:
"""
Remoção de linhas do dataset cujos characters (personagens) não 
são os principais e não há imagens (characters_unique['characters_unique_url']) armazenados
"""
list_characters_mains = characters_dataset['character'].unique().tolist()
characters_dataset_modified = characters_dataset.copy()
index = 0
for i in characters_dataset['relative']:
    if i not in list_characters_mains:
        characters_dataset_modified.drop(index, inplace=True)
    index +=1
characters_dataset_modified = characters_dataset_modified.reset_index()

In [ ]:
#Plotagens dos nodes e edges
sources = characters_dataset_modified['character']
targets = characters_dataset_modified['relative']
weights = characters_dataset_modified['relationship']

edge_data = zip(sources, targets, weights)
graph = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# permissão de manipulação interativa com o grafo criado
graph.barnes_hut()
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    
    graph.add_node(src, src, title=src, image=src, shape='circularImage')
    graph.add_node(dst, dst, title=dst, image=src, shape='circularImage')
    graph.add_edge(src, dst, value=w)

neighbor_map = graph.get_adj_list()

# adicão de novas características (relatives, image)
i = 0
for node in graph.nodes:
    node["title"] += " Relatives:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])
    
    for j in characters_unique.index:
        if node["id"] == characters_unique['character'][j]:
            node["image"] = characters_unique['characters_unique_url'][j]
    i+=1
    
graph.show("harryPotterCharacterNetwork.html")